# Pattern recognition CW2

In [8]:
from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')
train_idxs

#name format: x_label_camId_index.png

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Mar  6 20:42:54 2017',
 '__version__': '1.0',
 'camId': array([[1],
        [1],
        [1],
        ...,
        [2],
        [2],
        [2]], dtype=uint8),
 'filelist': array([[array(['1_001_1_01.png'], dtype='<U14')],
        [array(['1_001_1_02.png'], dtype='<U14')],
        [array(['1_001_1_03.png'], dtype='<U14')],
        ...,
        [array(['5_049_2_08.png'], dtype='<U14')],
        [array(['5_049_2_09.png'], dtype='<U14')],
        [array(['5_049_2_10.png'], dtype='<U14')]], dtype=object),
 'gallery_idx': array([[   21],
        [   23],
        [   24],
        ...,
        [14062],
        [14064],
        [14065]], dtype=uint16),
 'labels': array([[   1],
        [   1],
        [   1],
        ...,
        [1467],
        [1467],
        [1467]], dtype=uint16),
 'query_idx': array([[   22],
        [   27],
        [   49],
        ...,
        [14043],
        [14059],
       

In [9]:
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)initial commit 

In [13]:
import numpy as np
features_np = np.array(features)

In [16]:
features_np.shape

(14096, 2048)

In [178]:
train_idxs['filelist']

array([[array(['1_001_1_01.png'], dtype='<U14')],
       [array(['1_001_1_02.png'], dtype='<U14')],
       [array(['1_001_1_03.png'], dtype='<U14')],
       ...,
       [array(['5_049_2_08.png'], dtype='<U14')],
       [array(['5_049_2_09.png'], dtype='<U14')],
       [array(['5_049_2_10.png'], dtype='<U14')]], dtype=object)

In [70]:
train_idxs['labels'].shape

(14096, 1)

In [79]:
train_idxs['gallery_idx']

array([[   21],
       [   23],
       [   24],
       ...,
       [14062],
       [14064],
       [14065]], dtype=uint16)

In [38]:
train_idxs['train_idx'].shape

(7368, 1)

In [39]:
7368+5328

12696

In [40]:
train_idxs['query_idx'].shape

(1400, 1)

In [41]:
12696+1400

14096

In [44]:
features_np[0]

array([0.13791114, 1.12505555, 0.05404324, ..., 0.10747871, 0.04081609,
       0.68009406])

In [69]:
features_np[train_idxs['query_idx'].flatten()]

array([[0.98361254, 0.33195475, 0.26549727, ..., 0.02350602, 0.08491972,
        0.00514889],
       [1.30696166, 0.22219124, 0.62375975, ..., 0.02954952, 0.37348923,
        0.05303315],
       [0.29322624, 0.55588913, 0.06961903, ..., 0.15892459, 0.21527511,
        0.09551907],
       ...,
       [0.7593497 , 0.24537075, 2.19513273, ..., 0.47061139, 1.35971546,
        0.46198806],
       [0.48667279, 0.49618778, 0.30824399, ..., 0.04450084, 1.28799629,
        0.37683338],
       [0.05719076, 0.60635394, 0.58300596, ..., 0.40994745, 1.37061644,
        0.55975795]])

In [176]:
train_idxs['query_idx'][0][0]

22

In [174]:
img_name = train_idxs['filelist'][22][0][0]
[dc, label, camId, index] = img_name.split('_')
index = index[:-4]

In [175]:
gallery_images

array([[9.40880954e-01, 6.12001598e-01, 1.96013823e-01, ...,
        3.83011550e-02, 5.46127856e-02, 1.00457324e-02],
       [8.26966822e-01, 3.95451933e-01, 1.34190559e-01, ...,
        2.50734761e-02, 6.06398210e-02, 3.50554474e-02],
       [7.68881679e-01, 2.55869567e-01, 6.01241291e-01, ...,
        1.81159094e-01, 4.31653410e-01, 9.93161947e-02],
       ...,
       [4.79696468e-02, 4.94295180e-01, 6.11936390e-01, ...,
        7.43048429e-01, 1.20724857e+00, 6.64190292e-01],
       [7.24511072e-02, 5.16567469e-01, 5.90824187e-01, ...,
        6.04884207e-01, 1.05610693e+00, 7.29397178e-01],
       [9.02916938e-02, 9.08039138e-03, 1.26653552e+00, ...,
        2.80692708e-04, 4.21197325e-01, 8.77461396e-03]])

In [154]:
train_idxs['query_idx'].flatten()

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [307]:
from sklearn.neighbors import KNeighborsClassifier

def clean_gallery_imgs(gallery_images, gallery_names, label, camId):
    new_gallery_images = []
    new_gallery_labels = []
    for img, name in zip(gallery_images, gallery_names):
        [dc, label_g, camId_g, index] = str(name[0][0]).split('_')
        if [int(label_g), int(camId_g)] != [int(label), int(camId)]:
            new_gallery_images.append(img)
            new_gallery_labels.append(int(label_g))
    return new_gallery_images, new_gallery_labels

def delete_images(gallery_images, gallery_labels, query_image, query_index):
    
    gallery_names = train_idxs['filelist'][train_idxs['gallery_idx'].flatten()]
    query_name = str(train_idxs['filelist'][query_index][0][0])
    [dc, label, camId, index] = query_name.split('_')
    
    new_gallery_images, new_gallery_labels = clean_gallery_imgs(gallery_images, gallery_names, label, camId)
    return new_gallery_images, new_gallery_labels

In [308]:
from sklearn.neighbors import NearestNeighbors

def NNClassification_deletions(query_image, gallery_images, gallery_labels, query_index, k):
    neigh = NearestNeighbors(n_neighbors=k)
    #preparing the training data
    new_gallery_images, new_gallery_labels = delete_images(gallery_images, gallery_labels, query_image, query_index)
    neigh.fit(new_gallery_images)
    #neigh.kneighbors(query_image, 10)
    dist, ind = neigh.kneighbors(query_image, k)
    return dist, ind, new_gallery_labels #neigh.score(query_image, query_label), neigh.predict(query_image), 

In [ ]:
import time
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

query_images = features_np[train_idxs['query_idx'].flatten()]
gallery_images = features_np[train_idxs['gallery_idx'].flatten()]


def score_rank(query_label, rank_k, new_gallery_labels):
    rank_k_labels = np.array(new_gallery_labels)[rank_k.flatten()]
    return (query_label[0] in rank_k_labels)
list_of_truths = []
start = time.time()
for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
    if index == 100:
        acc = sum(list_of_truths)/len(list_of_truths)
        print(acc)
    
    if (index % 10 == 0):
        print(index, time.time() - start)
        
    #print(query_images[index], query_image)
    dist, rank_k, new_gallery_labels = NNClassification_deletions(query_images[index].reshape(1, 2048), gallery_images, gallery_labels, query_index, 1)
    score = score_rank(query_labels[index], rank_k, new_gallery_labels)
    #print(score)
    list_of_truths.append(score)
    
acc = sum(list_of_truths)/len(list_of_truths)
print(acc)

0 0.0
10 14.10928225517273
20 28.296353816986084
30 43.33415365219116
40 57.95706057548523
50 72.17009711265564
60 85.83188056945801
70 100.03488278388977
80 114.2319278717041
90 128.32824516296387
0.47
100 142.51335191726685
110 156.66947746276855
120 170.457617521286
130 184.15300345420837
140 198.1495885848999
150 210.87071180343628
160 223.93978071212769
170 237.50451850891113
180 250.86313223838806
190 264.27328181266785
200 277.8240556716919
210 290.8911352157593
220 305.46218156814575
230 318.407098531723
240 332.3318729400635
250 345.96742057800293
260 360.1475124359131
270 374.3595199584961
280 388.64337062835693
290 402.8114595413208
300 416.74919962882996
310 428.4609320163727
320 442.11140513420105
330 454.83339619636536
340 469.5904974937439
350 483.99100065231323
360 498.4832603931427
370 512.1008543968201
380 526.2151234149933
390 541.677784204483
400 555.038102388382
410 568.281491279602
420 582.3778057098389
430 596.4412109851837
440 609.1935794353485
450 623.082448482

In [ ]:
rank_k[0][0]

In [ ]:
train_idxs['query_idx'].flatten().tolist()

In [ ]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

x = NNClassification(train_idxs['query_idx'], train_idxs['gallery_idx'], gallery_labels, query_labels, 10)
print(x)

In [75]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
gallery_labels.shape

(5328, 1)